# ONE Quick Start
This tutorial will get you started searching and loading IBL data using Open Neurophysiology
Environment (ONE).

First we need to install ONE. If you don't already have IBL libraries, the easiest way is to run
 `pip install ONE-api`.

Now we need to import the ONE library and open a connection to the IBL public data server. To do
so, we create an ONE object, and ask it to connect to the IBL public server.

<div class="alert alert-info">
Info.

IBL internal users may use their Alyx credentials to access all IBL data.
[Click here](https://int-brain-lab.github.io/ONE/one_installation.html#connecting-to-specific-database-relevant-for-ibl-users) for details.
</div>

In [2]:
from one.api import ONE
ONE.setup(base_url='https://openalyx.internationalbrainlab.org', silent=True)
one = ONE(password='international')

Connected to https://openalyx.internationalbrainlab.org as user "intbrainlab"


Now we are going to search for an experiment to analyze. First let's find out what we can search by:

In [3]:
print(one.search_terms())

('task_protocol', 'datasets', 'name', 'offset', 'atlas_acronym', 'performance_gte', 'number', 'n_correct_trials', 'end_time', 'date_range', 'parent_session', 'type', 'narrative', 'qc', 'laboratory', 'project', 'extended_qc', 'dataset_types', 'atlas_id', 'django', 'procedures', 'auto_datetime', 'atlas_name', 'tag', 'histology', 'performance_lte', 'subject', 'limit', 'location', 'n_trials', 'dataset_qc_lte', 'start_time', 'projects', 'id', 'nickname', 'json', 'users')


Let's search for sessions recorded in August 2020, which contain a dataset 'probes.description.json',
meaning that electrophysiology was recorded. By adding the argument `details=True`, we get two
outputs - the experiment IDs uniquely identifying these sessions, and some information about the
experiments.

In [4]:
eids, info = one.search(date_range=['2020-08-01', '2020-08-31'], datasets='probes.description.json', details=True)

from pprint import pprint
pprint(eids)
pprint(info)

[UUID('b69b86be-af7d-4ecf-8cbf-0cd356afa1bd'),
 UUID('ebe2efe3-e8a1-451a-8947-76ef42427cc9'),
 UUID('edd22318-216c-44ff-bc24-49ce8be78374'),
 UUID('626126d5-eecf-4e9b-900e-ec29a17ece07'),
 UUID('71e55bfe-5a3a-4cba-bdc7-f085140d798e'),
 UUID('49e0ab27-827a-4c91-bcaa-97eea27a1b8d'),
 UUID('81a78eac-9d36-4f90-a73a-7eb3ad7f770b'),
 UUID('5adab0b7-dfd0-467d-b09d-43cb7ca5d59c'),
 UUID('e56541a5-a6d5-4750-b1fe-f6b5257bfe7c'),
 UUID('7f6b86f9-879a-4ea2-8531-294a221af5d0'),
 UUID('5d01d14e-aced-4465-8f8e-9a1c674f62ec'),
 UUID('8c33abef-3d3e-4d42-9f27-445e9def08f9'),
 UUID('c557324b-b95d-414c-888f-6ee1329a2329'),
 UUID('61e11a11-ab65-48fb-ae08-3cb80662e5d6'),
 UUID('c7248e09-8c0d-40f2-9eb4-700a8973d8c8'),
 UUID('280ee768-f7b8-4c6c-9ea0-48ca75d6b6f3'),
 UUID('ff48aa1d-ef30-4903-ac34-8c41b738c1b9'),
 UUID('03063955-2523-47bd-ae57-f7489dd40f15')]
[{'date': datetime.date(2020, 8, 19),
  'id': 'b69b86be-af7d-4ecf-8cbf-0cd356afa1bd',
  'lab': 'zadorlab',
  'number': 1,
  'projects': ['ibl_neuropixel_b

So there were four experiments matching the criteria in the public database. Now let's load the
probe information for the first experiment. `one.load_dataset` returns a single named dataset

In [5]:
eid = eids[0]  # Select the first experiment
probe_insertions = one.load_dataset(eid, 'probes.description')

print(f'N probes = {len(probe_insertions)}')
pprint(probe_insertions[0])

(S3) F:\FlatIron\openalyx.internationalbrainlab.org\zadorlab\Subjects\CSH_ZAD_026\2020-08-19\001\alf\probes.description.json: 100%|██████████| 482/482 [00:00<00:00, 1.66kB/s]

N probes = 2
{'label': 'probe00',
 'model': '3B2',
 'raw_file_name': 'D:/iblrig_data/Subjects/CSH_ZAD_026/2020-08-19/001/raw_ephys_data/_spikeglx_ephysData__g0/_spikeglx_ephysData__g0_imec0/_spikeglx_ephysData__g0_t0.imec0.ap.bin',
 'serial': 18194814382}


Now let’s see all the datasets associated with the first of these experiments. The command one.list_datasets returns the full path of all datasets, including the collection name and the extension. The ‘alf’ collection contains the preprocessed data we usually want to work with, and the data for each probe are in labeled subdirectories. We use the wildcard * because the data can be saved in different subdirectories for different spike sorters.

In [6]:
probe_label = probe_insertions[0]['label']
one.list_datasets(eid, collection=f'alf/{probe_label}*')

['alf/probe00/electrodeSites.brainLocationIds_ccf_2017.npy',
 'alf/probe00/electrodeSites.localCoordinates.npy',
 'alf/probe00/electrodeSites.mlapdv.npy',
 'alf/probe00/pykilosort/#2024-05-06#/_ibl_log.info_pykilosort.log',
 'alf/probe00/pykilosort/#2024-05-06#/_kilosort_whitening.matrix.npy',
 'alf/probe00/pykilosort/#2024-05-06#/_phy_spikes_subset.channels.npy',
 'alf/probe00/pykilosort/#2024-05-06#/_phy_spikes_subset.spikes.npy',
 'alf/probe00/pykilosort/#2024-05-06#/_phy_spikes_subset.waveforms.npy',
 'alf/probe00/pykilosort/#2024-05-06#/channels.brainLocationIds_ccf_2017.npy',
 'alf/probe00/pykilosort/#2024-05-06#/channels.labels.npy',
 'alf/probe00/pykilosort/#2024-05-06#/channels.localCoordinates.npy',
 'alf/probe00/pykilosort/#2024-05-06#/channels.mlapdv.npy',
 'alf/probe00/pykilosort/#2024-05-06#/channels.rawInd.npy',
 'alf/probe00/pykilosort/#2024-05-06#/clusters.amps.npy',
 'alf/probe00/pykilosort/#2024-05-06#/clusters.channels.npy',
 'alf/probe00/pykilosort/#2024-05-06#/clu

We might be interested in the data of this session that is not specific to the probe recording, e.g. the behavioural data or video data. We can find that if we list datasets in the alf collection without specifying the probe.

In [7]:
one.list_datasets(eid, collection='alf')

['alf/_ibl_bodyCamera.dlc.pqt',
 'alf/_ibl_bodyCamera.times.npy',
 'alf/_ibl_leftCamera.dlc.pqt',
 'alf/_ibl_leftCamera.features.pqt',
 'alf/_ibl_leftCamera.times.npy',
 'alf/_ibl_rightCamera.dlc.pqt',
 'alf/_ibl_rightCamera.features.pqt',
 'alf/_ibl_rightCamera.times.npy',
 'alf/_ibl_trials.goCueTrigger_times.npy',
 'alf/_ibl_trials.stimOff_times.npy',
 'alf/_ibl_trials.table.pqt',
 'alf/_ibl_wheel.position.npy',
 'alf/_ibl_wheel.timestamps.npy',
 'alf/_ibl_wheelMoves.intervals.npy',
 'alf/_ibl_wheelMoves.peakAmplitude.npy',
 'alf/bodyCamera.ROIMotionEnergy.npy',
 'alf/bodyROIMotionEnergy.position.npy',
 'alf/leftCamera.ROIMotionEnergy.npy',
 'alf/leftROIMotionEnergy.position.npy',
 'alf/licks.times.npy',
 'alf/probes.description.json',
 'alf/rightCamera.ROIMotionEnergy.npy',
 'alf/rightROIMotionEnergy.position.npy']

Let's load the preprocessed data associated with the left camera. There are two ways to do this.
`one.load_dataset` or `one.load_object`, which returns all the datasets with the same name part,
as an object. Let's use the second method to load all left camera data in the alf folder.

In [8]:
cam = one.load_object(eids[0], 'leftCamera', collection='alf')
cam.keys()

(S3) F:\FlatIron\openalyx.internationalbrainlab.org\zadorlab\Subjects\CSH_ZAD_026\2020-08-19\001\alf\_ibl_leftCamera.times.npy: 100%|██████████| 2.61M/2.61M [00:01<00:00, 2.19MB/s]
(S3) F:\FlatIron\openalyx.internationalbrainlab.org\zadorlab\Subjects\CSH_ZAD_026\2020-08-19\001\alf\leftCamera.ROIMotionEnergy.npy: 100%|██████████| 2.61M/2.61M [00:00<00:00, 5.70MB/s]
(S3) F:\FlatIron\openalyx.internationalbrainlab.org\zadorlab\Subjects\CSH_ZAD_026\2020-08-19\001\alf\_ibl_leftCamera.dlc.pqt: 100%|██████████| 47.2M/47.2M [00:03<00:00, 13.1MB/s]
(S3) F:\FlatIron\openalyx.internationalbrainlab.org\zadorlab\Subjects\CSH_ZAD_026\2020-08-19\001\alf\_ibl_leftCamera.features.pqt: 100%|██████████| 15.4k/15.4k [00:00<00:00, 57.5kB/s]


dict_keys(['times', 'ROIMotionEnergy', 'dlc', 'features'])

To load only specific data associated with this object, we can use the attribute keyword. Let's load the times and the DLC traces.

In [9]:
cam = one.load_object(eids[0], 'leftCamera', collection='alf', attribute=['times', 'dlc'])
cam.keys()

dict_keys(['times', 'dlc'])

And that's the end of the quick start tutorial! For more information on any of these commands you
can use the standard help function:

In [10]:
help(one.list_datasets)

Help on method list_datasets in module one.api:

list_datasets(eid=None, filename=None, collection=None, revision=None, qc=<QC.FAIL: 40>, ignore_qc_not_set=False, details=False, query_type=None, default_revisions_only=False, keep_eid_index=False) -> Union[numpy.ndarray, pandas.core.frame.DataFrame] method of one.api.OneAlyx instance
    Given an eid, return the datasets for those sessions.
    
    If no eid is provided, a list of all datasets is returned.  When details is false, a sorted
    array of unique datasets is returned (their relative paths).
    
    Parameters
    ----------
    eid : str, UUID, pathlib.Path, dict
        Experiment session identifier; may be a UUID, URL, experiment reference string
        details dict or Path.
    filename : str, dict, list
        Filters datasets and returns only the ones matching the filename.
        Supports lists asterisks as wildcards.  May be a dict of ALF parts.
    collection : str, list
        The collection to which the objec

For detailed tutorials, guides and examples, [click here for the full ONE API documentation
Website](https://int-brain-lab.github.io/ONE/).